# Inception-v4

In [ ]:
%conda install -y gdown

In [ ]:
import gdown
import zipfile
from pathlib import Path

from tensorflow.config import list_physical_devices
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Input, Layer, Conv2D, MaxPool2D, AvgPool2D, Rescaling, GlobalAveragePooling2D, Dropout, Flatten, Dense, concatenate
from tensorflow.keras import Model

In [ ]:
zip_name = "dataset.zip"
wd = Path("/kaggle/working")
extract_path = Path(wd, "data")
class_names = list()

train_path = Path(extract_path, "train")
test_path = Path(extract_path, "test")
model_path = Path(wd, "models/final_models/inception4.h5")

## Model settings

In [ ]:
batch_size = 64
img_size = 299
epochs = 100
seed = 27
validation_split = 0.2

## Download dataset

In [ ]:
gdown.download(
    "https://drive.google.com/uc?id=18_MDbhjncjKGwa1N9zYYUmiFzZYxzF-6",
    zip_name
)

zip_ref = zipfile.ZipFile(Path(wd, zip_name), 'r')
zip_ref.extractall(extract_path)
zip_ref.close()

## Get class names from the training directory

In [ ]:
class_names = [class_name.name for class_name in train_path.iterdir()]
for class_name in class_names:
    print(class_name)

## Check number of GPUs

In [ ]:
print("Num GPUs Available: {}".format(len(list_physical_devices('GPU'))))

## Load training and validation data

In [ ]:
training_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='training',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

validation_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='validation',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

## Define Inception-v4 model parts

In [ ]:
class Stem(Layer):
    def __init__(self):
        super(Stem, self).__init__()
        self.conv1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')
        self.conv2 = Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu')
        self.conv3 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')

        self.maxpool4a = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')
        self.conv4b = Conv2D(filters=96, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')

        self.conv5a = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv5b = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')

        self.conv6a = Conv2D(filters=96, kernel_size=(3, 3), padding='valid', activation='relu')
        self.conv6b = Conv2D(filters=64, kernel_size=(7, 1), padding='same', activation='relu')

        self.conv7b = Conv2D(filters=64, kernel_size=(1, 7), padding='same', activation='relu')

        self.conv8b = Conv2D(filters=96, kernel_size=(3, 3), padding='valid', activation='relu')

        self.conv9a = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')
        self.maxpool9b = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)

        x0 = self.maxpool4a(x)
        x1 = self.conv4b(x)

        x = concatenate([x0, x1], axis=-1)

        x0 = self.conv5a(x)
        x0 = self.conv6a(x0)

        x1 = self.conv5b(x)
        x1 = self.conv6b(x1)
        x1 = self.conv7b(x1)
        x1 = self.conv8b(x1)

        x = concatenate([x0, x1], axis=-1)

        x0 = self.conv9a(x)
        x1 = self.maxpool9b(x)

        return concatenate([x0, x1], axis=-1)

    def get_config(self):
        return super(Stem, self).get_config()

In [ ]:
class InceptionA(Layer):
    def __init__(self):
        super(InceptionA, self).__init__()
        self.avgpool1a = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv1b = Conv2D(filters=96, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1c = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1d = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')

        self.conv2a = Conv2D(filters=96, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2c = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')
        self.conv2d = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')

        self.conv3d = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')

    def call(self, inputs):
        x0 = self.avgpool1a(inputs)
        x0 = self.conv2a(x0)

        x1 = self.conv1b(inputs)

        x2 = self.conv1c(inputs)
        x2 = self.conv2c(x2)

        x3 = self.conv1d(inputs)
        x3 = self.conv2d(x3)
        x3 = self.conv3d(x3)

        return concatenate([x0, x1, x2, x3], axis=-1)

    def get_config(self):
        return super(InceptionA, self).get_config()

In [ ]:
class InceptionB(Layer):
    def __init__(self):
        super(InceptionB, self).__init__()
        self.avgpool1a = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv1b = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1c = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1d = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')

        self.conv2a = Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2c = Conv2D(filters=224, kernel_size=(1, 7), padding='same', activation='relu')
        self.conv2d = Conv2D(filters=192, kernel_size=(1, 7), padding='same', activation='relu')

        self.conv3c = Conv2D(filters=256, kernel_size=(1, 7), padding='same', activation='relu')
        self.conv3d = Conv2D(filters=224, kernel_size=(7, 1), padding='same', activation='relu')

        self.conv4d = Conv2D(filters=224, kernel_size=(1, 7), padding='same', activation='relu')

        self.conv5d = Conv2D(filters=256, kernel_size=(7, 1), padding='same', activation='relu')


    def call(self, inputs):
        x0 = self.avgpool1a(inputs)
        x0 = self.conv2a(x0)

        x1 = self.conv1b(inputs)

        x2 = self.conv1c(inputs)
        x2 = self.conv2c(x2)
        x2 = self.conv3c(x2)

        x3 = self.conv1d(inputs)
        x3 = self.conv2d(x3)
        x3 = self.conv3d(x3)
        x3 = self.conv4d(x3)
        x3 = self.conv5d(x3)

        return concatenate([x0, x1, x2, x3], axis=-1)


    def get_config(self):
        return super(InceptionB, self).get_config()

In [ ]:
class InceptionC(Layer):
    def __init__(self):
        super(InceptionC, self).__init__()
        self.avgpool1a = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv1b = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1c = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv1d = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')

        self.conv2a = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2c_1 = Conv2D(filters=256, kernel_size=(1, 3), padding='same', activation='relu')
        self.conv2c_2 = Conv2D(filters=256, kernel_size=(3, 1), padding='same', activation='relu')
        self.conv2d = Conv2D(filters=448, kernel_size=(1, 3), padding='same', activation='relu')

        self.conv3d = Conv2D(filters=512, kernel_size=(3, 1), padding='same', activation='relu')
        
        self.conv4d_1 = Conv2D(filters=256, kernel_size=(3, 1), padding='same', activation='relu')
        self.conv4d_2 = Conv2D(filters=256, kernel_size=(1, 3), padding='same', activation='relu')

    def call(self, inputs):
        x0 = self.avgpool1a(inputs)
        x0 = self.conv2a(x0)

        x1 = self.conv1b(inputs)

        x2 = self.conv1c(inputs)
        x2_0 = self.conv2c_1(x2)
        x2_1 = self.conv2c_2(x2)
        x2 = concatenate([x2_0, x2_1], axis=-1)

        x3 = self.conv1d(inputs)
        x3 = self.conv2d(x3)
        x3 = self.conv3d(x3)
        x3_0 = self.conv4d_1(x3)
        x3_1 = self.conv4d_2(x3)
        x3 = concatenate([x3_0, x3_1], axis=-1)

        return concatenate([x0, x1, x2, x3], axis=-1)

    def get_config(self):
        return super(InceptionC, self).get_config()

In [ ]:
class ReductionA(Layer):
    def __init__(self):
        super(ReductionA, self).__init__()
        self.maxpool1a = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')

        self.conv1b = Conv2D(filters=384, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')

        self.conv1c = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2c = Conv2D(filters=224, kernel_size=(3, 3), padding='same', activation='relu')
        self.conv3c = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')

    def call(self, inputs):
        x0 = self.maxpool1a(inputs)

        x1 = self.conv1b(inputs)

        x2 = self.conv1c(inputs)
        x2 = self.conv2c(x2)
        x2 = self.conv3c(x2)

        return concatenate([x0, x1, x2], axis=-1)

    def get_config(self):
        return super(ReductionA, self).get_config()

In [ ]:
class ReductionB(Layer):
    def __init__(self):
        super(ReductionB, self).__init__()
        self.maxpool1a = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')

        self.conv1b = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2b = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')

        self.conv1c = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2c = Conv2D(filters=256, kernel_size=(1, 7), padding='same', activation='relu')
        self.conv3c = Conv2D(filters=320, kernel_size=(7, 1), padding='same', activation='relu')
        self.conv4c = Conv2D(filters=320, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')

    def call(self, inputs):
        x0 = self.maxpool1a(inputs)

        x1 = self.conv1b(inputs)
        x1 = self.conv2b(x1)

        x2 = self.conv1c(inputs)
        x2 = self.conv2c(x2)
        x2 = self.conv3c(x2)
        x2 = self.conv4c(x2)

        return concatenate([x0, x1, x2], axis=-1)

    def get_config(self):
        return super(ReductionB, self).get_config()

In [ ]:
def Inception4():
    inputs = Input(shape=(img_size, img_size, 3))

    x = Stem()(inputs)

    for _ in range(4):
        x = InceptionA()(x)

    x = ReductionA()(x)

    for _ in range(7):
        x = InceptionB()(x)

    x = ReductionB()(x)
    
    for _ in range(3):
        x = InceptionC()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.2)(x)
    x = Flatten()(x)
    x = Dense(units=len(class_names), activation='softmax')(x)

    return Model(inputs=inputs, outputs=x, name='Inception-v4')

In [ ]:
model = Inception4()

model.compile(
    optimizer=SGD(learning_rate=0.001, momentum=0.9),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

plot_model(
    model, 
    to_file='inception_v4.png', 
    show_shapes=True,
    show_layer_names=True
)